In [1]:
import keras
import csv
import os
import re
import sys
import numpy as np
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, Flatten, Activation, Lambda
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalAvgPool1D
from keras.models import Model, Sequential
from keras import backend as K


from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing, svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.pipeline import Pipeline

module_path = os.path.abspath(os.path.join('../embedding-classif/'))
if module_path not in sys.path:
    sys.path.append(module_path)

# my stuff in the helpers/ directory
from helpers import files_helper, texts_helper, metrics_helper, tags_helper

Using TensorFlow backend.


In [2]:
texts,labels_index,labels_0 = files_helper.read_20_newgroup_files()       

In [3]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100
LABELS_MIN_DOC_COUNT = int(10)
BATCH_SIZE=32
NUM_EPOCHS=10

In [4]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                     filters='\'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [5]:
[inverse_word_index[idx] for idx in sequences[0]]

['archive',
 'name',
 'atheism',
 'resources',
 'alt',
 'atheism',
 'archive',
 'name',
 'resources',
 'last',
 'modified',
 '11',
 'december',
 '1992',
 'version',
 '1',
 '0',
 'atheist',
 'resources',
 'addresses',
 'of',
 'atheist',
 'organizations',
 'usa',
 'freedom',
 'from',
 'religion',
 'foundation',
 'darwin',
 'fish',
 'bumper',
 'stickers',
 'and',
 'assorted',
 'other',
 'atheist',
 'are',
 'available',
 'from',
 'the',
 'freedom',
 'from',
 'religion',
 'foundation',
 'in',
 'the',
 'us',
 'write',
 'to',
 'p',
 'o',
 'box',
 '750',
 'madison',
 'wi',
 'telephone',
 '608',
 '256',
 '8900',
 'evolution',
 'designs',
 'evolution',
 'designs',
 'sell',
 'the',
 'darwin',
 'fish',
 'it',
 's',
 'a',
 'fish',
 'symbol',
 'like',
 'the',
 'ones',
 'christians',
 'stick',
 'on',
 'their',
 'cars',
 'but',
 'with',
 'feet',
 'and',
 'the',
 'word',
 'darwin',
 'written',
 'inside',
 'the',
 'deluxe',
 '3d',
 'plastic',
 'fish',
 'is',
 '4',
 '95',
 'in',
 'the',
 'us',
 'write',


In [6]:
tokenized_texts = []

for seq in sequences:
    truncated_seq = seq[:MAX_SEQUENCE_LENGTH]
    tokenized_txt = " ".join([inverse_word_index[idx] for idx in truncated_seq])
    tokenized_texts.append(tokenized_txt)

In [7]:
data = tokenized_texts
indices = np.arange(len(data))
np.random.shuffle(indices)

labels_1 = to_categorical(np.asarray(labels_0))

data = [data[i] for i in indices]
labels_2 = labels_1[indices]
num_validation_samples = int(VALIDATION_SPLIT * len(data))

data_train = data[:-num_validation_samples]
data_val = data[-num_validation_samples:]


In [8]:
# TfidfVectorizer = CountVectorizer + TFidfTransformer

# does this configure snooping?
# of course, I'm fitting on the whole data...
vect = TfidfVectorizer(max_features=MAX_NB_WORDS).fit(data)
X_train = vect.transform(data_train)
X_val = vect.transform(data_val)

sc = StandardScaler()

X_train = X_train.todense()
X_val = X_val.todense()

Y_train = labels_2[:-num_validation_samples]
Y_val = labels_2[-num_validation_samples:]

X_train.shape,X_val.shape,Y_train.shape,Y_val.shape

((15998, 19860), (3999, 19860), (15998, 20), (3999, 20))

In [ ]:
sess = tf.Session()

def custom_objective(y_true, y_pred):
    
    print(keras.backend.shape(y_true))
    first_log = K.log(y_pred)    
    first_log = first_log * y_true
    second_log = K.log(1 - y_pred)
    second_log = second_log * (1 - y_true)
    result = second_log + first_log
    return (-1 * K.mean(result)) 

In [31]:
# shape: A shape tuple (integer), not including the batch size.
#     For instance, `shape=(32,)` indicates that the expected input
#     will be batches of 32-dimensional vectors.
# batch_shape: A shape tuple (integer), including the batch size.
#     For instance, `batch_shape=(10, 32)` indicates that
#     the expected input will be batches of 10 32-dimensional vectors.
#     `batch_shape=(None, 32)` indicates batches of an arbitrary number
#     of 32-dimensional vectors.

inputs = Input(shape=(19860,))

x = Dense(64,activation='relu')(inputs)
x = Dropout(0.5)(x)
x = Dense(64,activation='sigmoid')(x)
x = Dropout(0.5)(x)
preds = Dense(20,activation='sigmoid')(x)

model = Model(inputs=inputs,outputs=preds)
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['acc'])

In [32]:
%%time

model.fit(X_train,Y_train, validation_data=(X_val, Y_val),
         epochs=10, batch_size=32)

Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 6s - loss: 0.2463 - acc: 0.9311 - val_loss: 0.1904 - val_acc: 0.9500
Epoch 2/10
15998/15998 [==============================] - 6s - loss: 0.1845 - acc: 0.9501 - val_loss: 0.1505 - val_acc: 0.9500
Epoch 3/10
15998/15998 [==============================] - 6s - loss: 0.1478 - acc: 0.9519 - val_loss: 0.1129 - val_acc: 0.9542
Epoch 4/10
15998/15998 [==============================] - 6s - loss: 0.1184 - acc: 0.9573 - val_loss: 0.0871 - val_acc: 0.9636
Epoch 5/10
15998/15998 [==============================] - 6s - loss: 0.0978 - acc: 0.9639 - val_loss: 0.0709 - val_acc: 0.9718
Epoch 6/10
15998/15998 [==============================] - 6s - loss: 0.0831 - acc: 0.9692 - val_loss: 0.0605 - val_acc: 0.9775
Epoch 7/10
15998/15998 [==============================] - 6s - loss: 0.0721 - acc: 0.9736 - val_loss: 0.0539 - val_acc: 0.9815
Epoch 8/10
15998/15998 [==============================] - 6s -